# Homework 2

# Set up

## Installing packages

In [37]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai
!pip install -q "langgraph"

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [38]:
from google.colab import userdata
# GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [41]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 3.54MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 2.64MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 3.07MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 2.91MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 3.35MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [42]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [43]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [52]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
# llm = ChatGoogleGenerativeAI(
#     model="gemini-3-flash-preview",
#     google_api_key=GEMINI_VERTEX_API_KEY,
#     temperature=0,
# )

from langchain_openai import ChatOpenAI
DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
llm = ChatOpenAI(
    model="deepseek-chat",          # or "deepseek-reasoner"
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com/v1",
    temperature=0,
)

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content="I'll help you with that! First, let me say hello to Bao, and then search for Alice on Facebook." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 2940, 'total_tokens': 3008, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 2880}, 'prompt_cache_hit_tokens': 2880, 'prompt_cache_miss_tokens': 60}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '9117f430-4133-4983-88c9-8a21758f0707', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c90be-c7cd-70a2-902e-4a9652dab63c-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'call_00_BqLqH8zb4rDwIoJwYPm5X5QG', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2940, 'output_tokens': 68, 'total_tokens': 3008, 'input_token_details': {'cache_read': 2880}, 'output_token_details': {}}


In [53]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
await tools[5].ainvoke({'person_id': 456})

[{'type': 'text',
  'text': '{"profile_id":456,"post_count":4,"total_likes":5,"liked_by":[4390,3622,7500,4269,8464],"engagement_score":1.25}',
  'id': 'lc_c4a9ef1f-54b0-4162-a8f0-a2bc3ab2ca57'}]

In [60]:
"""
Agentic CV Audit System
-----------------------
Description: Automated background check system using ReAct agents and MCP social graph tools.
Architecture:
- Orchestration: LangGraph / LangChain Agents
- Validation Logic: 6-Phase cross-referencing (LinkedIn vs Facebook vs CV)
- Error Handling: Tool output serialization and regex-based score fallback
"""

import re
import json
import asyncio
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.tools import StructuredTool

# 适配不同版本的 LangGraph/LangChain
try:
    from langgraph.prebuilt import create_react_agent
except ImportError:
    from langchain.agents import create_agent as create_react_agent

# 1. CORE INSTRUCTIONS (Prompt Engineering)

CV_AUDIT_PROMPT = """You are an elite Background Integrity Auditor.
Your objective is to validate a candidate's CV by cross-referencing claims against
their LinkedIn and Facebook footprints via the provided SocialGraph MCP tools.

### OPERATIONAL PHASES (Execute sequentially):
PHASE 1: DATA EXTRACTION - Pull Name, City/Country, Education History, and Work Timeline from CV.
PHASE 2: LINKEDIN DISCOVERY - Search via 'search_linkedin_people'. If initial search fails, use fuzzy matching.
PHASE 3: PROFESSIONAL DEEP-DIVE - Fetch profile details for the top match. Focus on experience/tenure.
PHASE 4: SOCIAL CROSS-CHECK - Search Facebook users to find the candidate's personal profile.
PHASE 5: SOCIAL DATA FETCH - Retrieve Facebook profile info (current company, city, education).
PHASE 6: DISCREPANCY ANALYSIS - Compare CV vs. LinkedIn vs. Facebook.

### DATA SOURCE WEIGHTING:
- LinkedIn: Primary Truth Source. Direct contradictions here are SEVERE issues.
- Facebook: Secondary Context. Professional data here might be stale; treat mismatches as MINOR unless LinkedIn concurs.
- Corroboration Principle: If LinkedIn and the CV agree on an 'odd' detail (e.g., a short gap), mark it as VERIFIED.

### RATING SCALE:
0.90 - 1.00: Full alignment across platforms.
0.70 - 0.89: LinkedIn matches CV; only minor/explainable Facebook deviations.
0.50 - 0.69: Partial LinkedIn match; noticeable timeline or role discrepancies.
0.30 - 0.49: LinkedIn contradicts CV significantly (wrong company, wrong years).
0.00 - 0.29: Major integrity failure or obvious identity fraud.

### DECISION THRESHOLD:
- Score > 0.65 -> VERDICT: VERIFIED
- 0.40 <= Score <= 0.65 -> VERDICT: SUSPICIOUS
- Score < 0.40 -> VERDICT: FRAUDULENT

### MANDATORY REPORTING TEMPLATE:
Finalize your analysis with this exact structure:

---VERIFICATION SUMMARY---
CREDIT_SCORE: [Value 0.0-1.0]
IDENTIFIED_ISSUES:
- [Item 1 or "None detected"]
- [Item 2]
FINAL_VERDICT: [VERIFIED / SUSPICIOUS / FRAUDULENT]
SUMMARY_EXPLANATION: [2-3 sentences max]
---END OF REPORT---
"""

# 2. UTILITIES & TOOL WRAPPERS

def stringify_tool_output(target_tool):
    """
    Ensures MCP tool outputs are JSON-serialized strings.
    Prevents API errors caused by sending raw lists/dicts to model message history.
    """
    def _to_json_str(data):
        return data if isinstance(data, str) else json.dumps(data, ensure_ascii=False, default=str)

    async def _wrapped_acall(*args, **kwargs):
        payload = args[0] if args and isinstance(args[0], dict) else kwargs
        output = await target_tool.ainvoke(payload)
        return _to_json_str(output)

    schema = getattr(target_tool, "args_schema", None) or getattr(target_tool, "args", None)

    return StructuredTool(
        name=target_tool.name,
        description=target_tool.description,
        args_schema=schema,
        coroutine=_wrapped_acall,
        func=lambda *a, **k: _to_json_str(target_tool.invoke(a[0] if a and isinstance(a[0], dict) else k))
    )

def prepare_agent_tools(mcp_list):
    """Returns a list of tools with stringified outputs."""
    return [stringify_tool_output(t) for t in mcp_list]

# 3. SCORE EXTRACTION LOGIC

def get_reliability_score(raw_text: str) -> float:
    """
    Parses the credit score from the agent's text.
    Uses a three-stage fallback: Structured block -> Global regex -> Keyword inference.
    """
    # Stage 1: Check the formatted summary block
    summary_block = re.search(r'---VERIFICATION SUMMARY---(.*?)---END OF REPORT---', raw_text, re.S | re.I)
    if summary_block:
        val = re.search(r'CREDIT_SCORE\s*:\s*([0-9]*\.?[0-9]+)', summary_block.group(1), re.I)
        if val:
            return max(0.0, min(1.0, float(val.group(1))))

    # Stage 2: Look for 'Score' pattern anywhere
    general_match = re.search(r'SCORE\s*[:]\s*([0-9]*\.?[0-9]+)', raw_text, re.I)
    if general_match:
        val = float(general_match.group(1))
        if 0.0 <= val <= 1.0: return val

    # Stage 3: Infer based on keywords
    txt = raw_text.upper()
    if 'FRAUDULENT' in txt: return 0.20
    if 'SUSPICIOUS' in txt: return 0.45
    if 'VERIFIED' in txt: return 0.90

    return 0.5

async def llm_score_recalibration(text_content: str, model) -> float:
    """Secondary check: Uses LLM to re-evaluate if the regex result is ambiguous."""
    refinement_prompt = (
        "Analyze the following CV verification report. Extract the overall reliability "
        "score as a single decimal between 0.0 and 1.0. Output the number ONLY.\n\n"
        f"Report excerpt:\n{text_content[-2500:]}"
    )
    try:
        res = await model.ainvoke([HumanMessage(content=refinement_prompt)])
        score_val = re.search(r'([0-9]*\.?[0-9]+)', res.content)
        if score_val:
            v = float(score_val.group(1))
            return v if 0.0 <= v <= 1.0 else 0.5
    except:
        pass
    return 0.5

# 4. EXECUTION ENGINE

async def run_cv_integrity_check(cv_text: str, mcp_tools_raw, llm_instance, idx: int):
    """Orchestrates the ReAct agent for a single CV audit."""

    agent_tools = prepare_agent_tools(mcp_tools_raw)

    # 构建 Agent (适配不同库版本)
    try:
        # LangGraph 风格
        agent_brain = create_react_agent(llm_instance, agent_tools, prompt=CV_AUDIT_PROMPT)
        input_data = {"messages": [HumanMessage(content=f"Begin Audit for CV #{idx+1}:\n\n{cv_text}")]}
    except TypeError:
        # 备选风格
        agent_brain = create_react_agent(llm_instance, agent_tools)
        input_data = {"messages": [SystemMessage(content=CV_AUDIT_PROMPT),
                                   HumanMessage(content=f"Begin Audit for CV #{idx+1}:\n\n{cv_text}")]}

    try:
        execution_path = await agent_brain.ainvoke(input_data, config={"recursion_limit": 50})

        # 寻找 AI 最后的非工具调用回复
        history = execution_path.get("messages", [])
        final_msg = ""
        for msg in reversed(history):
            if isinstance(msg, AIMessage) and not msg.tool_calls:
                final_msg = msg.content
                break

        if not final_msg and history:
            final_msg = history[-1].content

        score = get_reliability_score(final_msg)
        return {"index": idx, "score": score, "raw_report": final_msg}

    except Exception as err:
        print(f"Error auditing CV_{idx+1}: {err}")
        return {"index": idx, "score": 0.5, "raw_report": f"System error: {err}"}

# 5. MAIN BATCH PROCESSING

cv_audit_results = []

for i, cv_item in enumerate(all_cvs):
    print(f"\n[{i+1}/{len(all_cvs)}] Auditing: {cv_item['file']}...")

    audit_record = await run_cv_integrity_check(
        cv_text=cv_item["text"],
        mcp_tools_raw=mcp_tools,
        llm_instance=llm,
        idx=i
    )

    # 如果正则没抓到明确分数，启动 LLM 提取
    if audit_record["score"] == 0.5 and "CREDIT_SCORE" not in audit_record["raw_report"].upper():
        print(f"-> Regex uncertain, invoking LLM extractor for CV_{i+1}...")
        audit_record["score"] = await llm_score_recalibration(audit_record["raw_report"], llm)

    cv_audit_results.append(audit_record)

# 最终分数值列表用于评估
scores = [res["score"] for res in cv_audit_results]

print("\n" + "="*50)
print("BATCH AUDIT COMPLETE")
print(f"Scores Generated: {scores}")
print("="*50)

# 6. FINAL DISPLAY

for result in cv_audit_results:
    status = "RELIABLE" if result['score'] > 0.5 else "UNRELIABLE/FAIL"
    print(f"\nCV_{result['index']+1}: Score {result['score']:.2f} -> {status}")


[1/5] Auditing: CV_1.pdf...


/tmp/ipython-input-1803725662.py:151: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_brain = create_react_agent(llm_instance, agent_tools, prompt=CV_AUDIT_PROMPT)



[2/5] Auditing: CV_2.pdf...

[3/5] Auditing: CV_3.pdf...

[4/5] Auditing: CV_4.pdf...

[5/5] Auditing: CV_5.pdf...

BATCH AUDIT COMPLETE
Scores Generated: [0.7, 0.9, 0.75, 0.3, 0.25]

CV_1: Score 0.70 -> RELIABLE

CV_2: Score 0.90 -> RELIABLE

CV_3: Score 0.75 -> RELIABLE

CV_4: Score 0.30 -> UNRELIABLE/FAIL

CV_5: Score 0.25 -> UNRELIABLE/FAIL


# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [61]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [67]:
scores      = [res["score"] for res in cv_audit_results]
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)
print(scores)

{'decisions': [1, 1, 1, 0, 0], 'correct': 5, 'total': 5, 'final_score': 1.0}
[0.7, 0.9, 0.75, 0.3, 0.25]
